In [10]:
import pandas as pd
file_names = {
    "primary_cohort": "data/s41598-020-73558-3_sepsis_survival_primary_cohort.csv",
    "study_cohort": "data/s41598-020-73558-3_sepsis_survival_study_cohort.csv",
    "validation_cohort": "data/s41598-020-73558-3_sepsis_survival_validation_cohort.csv",
}
dict_df = dict()
for key, file in file_names.items():
    dict_df[key] = pd.read_csv(file)

### **Dataset information**

#### **What do the instances in this dataset represent?**

For the primary cohort, they represent records of patients affected by sepsis potential preconditions (ante Sepsis-3 definition); for the study cohort, they represent only the patients’ admissions defined by the novel Sepsis-3 definition.

#### **Are there recommended data splits?**

No recommendation, standard train-test split could be used. Can use three-way holdout split (i.e., training, validation/development, testing) when doing model selection.

#### **Does the dataset contain data that might be considered sensitive in any way?**

Yes. It contains information about the gender and age of the patient.

#### **Was there any data preprocessing performed?**

All the categorical variables have been encoded (so no preprocessing is necessary).

#### **Additional Information**

Primary cohort from Norway:
- 4 features for 110,204 patient admissions
- file: 's41598-020-73558-3_sepsis_survival_primary_cohort.csv'

Study cohort (a subset of the primary cohort) from Norway:
- 4 features for 19,051 patient admissions
- file: 's41598-020-73558-3_sepsis_survival_study_cohort.csv'

Validation cohort from South Korea:
- 4 features for 137 patients
- file: 's41598-020-73558-3_sepsis_survival_validation_cohort.csv'

The validation cohort from South Korea was used by Chicco and Jurman (2020) as an external validation cohort to confirm the generalizability of their proposed approach. 

#### **Has Missing Values?**

No

In [12]:
dict_df["primary_cohort"]

,age_years,sex_0male_1female,episode_number,hospital_outcome_1alive_0dead
0,21,1,1,1
1,20,1,1,1
2,21,1,1,1
3,77,0,1,1
4,72,0,1,1
...,...,...,...,...
110199,0,0,1,1
110200,0,1,1,1
110201,70,1,1,1
110202,0,0,1,1


In [13]:
dict_df["study_cohort"]

,age_years,sex_0male_1female,episode_number,hospital_outcome_1alive_0dead
0,7,1,1,1
1,17,0,2,1
2,70,0,1,1
3,76,0,1,1
4,8,0,1,1
...,...,...,...,...
19046,33,1,1,1
19047,58,0,1,1
19048,44,0,2,1
19049,61,0,3,0


In [14]:
dict_df["validation_cohort"]

,age_years,sex_0male_1female,episode_number,hospital_outcome_1alive_0dead
0,20,0,1,1
1,22,0,1,1
2,26,1,2,0
3,33,1,1,1
4,33,0,1,1
...,...,...,...,...
132,47,0,1,1
133,50,0,1,0
134,62,0,1,1
135,58,0,1,0
